In [5]:
import math
import numpy as np
import pandas as pd
import sklearn.model_selection
import matplotlib.pyplot as plt
from sklearn import preprocessing
from collections import defaultdict, Counter
from sklearn.neighbors import NearestNeighbors
from sklearn import datasets, linear_model,cluster
from sklearn.metrics import mean_squared_error, r2_score 

import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

### Raw Data Reading 

In [6]:
raw_df = pd.read_csv("zip_housing_house.csv")
print("Raw data has: " + str(len(raw_df)) + " rows")

Raw data has: 91644 rows


#### Raw Data Processing: Drop all rows where (beds, baths_full, lot_size, building_size) has na, and fill baths_half with 0

In [7]:
# Deleting unreasonable data
dropped_df = raw_df.dropna(subset=['beds','baths_full']) 
dropped_df[['baths_half']] = dropped_df[['baths_half']].fillna(0)
dropped_df[['garage']] = dropped_df[['garage']].fillna(0)
count = 0
df = pd.DataFrame()
grouped_df = dropped_df.groupby("postal_code")
for name, group in grouped_df:
    group['lot_size'].fillna(group['lot_size'].median(), inplace=True)
    group['building_size'].fillna(group['building_size'].median(), inplace=True)
    ##group[["lot_size"]].fillna(group[["lot_size"]].median(), inplace = True)
    ##group[["building_size"]].fillna(group[["building_size"]].median(), inplace = True)
    group
    if(count == 0):
        df = group
        count += 1
    else:
        df = pd.concat([df, group])
    ##print("After: " + str(len(group)))

df.dropna(subset=['lot_size','building_size'], inplace = True)
print("Dropped data has: " + str(len(df)) + " rows")

d:\python3\lib\site-packages\numpy\lib\nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


Dropped data has: 86806 rows


Since our dataset is large enough, and we do not have much professional knowledge about property markets, we decided to ignore all rows that contain null values instead of filling in estimate values (mean or median).

## DBSCAN Method

#### Normalizing

In [5]:
scaled_dataset = dropped_df[dropped_df.columns[4:11]]
scaled_dataset = preprocessing.normalize(scaled_dataset)
scaled_dataset = pd.DataFrame(scaled_dataset)
scaled_dataset.head()

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

#### Finding Best DBSCAN (eps and Minpoints)

Minpoints is suggested as 2*dimension which should be 14 in our case

In [ ]:
neighbors = NearestNeighbors(n_neighbors = 14)
neighbors_fit = neighbors.fit(scaled_dataset)
distances, indices = neighbors_fit.kneighbors(scaled_dataset)
distances = np.sort(distances, axis=0)
distances = distances[:,1]

#### Zoom in to find 'elbow' optimization point

In [ ]:
plt.plot(distances)
plt.axis([74000, 80000, 0.0002, 0.0010])

#### Fit DBSCAN Model

In [ ]:
minpts = 14
e = 0.0006
db = cluster.DBSCAN(eps=e,min_samples=minpts, metric='euclidean', 
                    metric_params=None, algorithm='auto', 
                    leaf_size=30, p=None, n_jobs=None)

#### Get Labels: 0 for 'not outliers'; 1 for 'outliers'

In [ ]:
model = db.fit(scaled_dataset)
labels = []
outliers = 0
for i in model.labels_:
    if i == -1:
        labels.append(1)
        outliers += 1
    else:
        labels.append(0)
print("Found total: " + str(outliers) + " outliers.")

#### Combine Outliers and Processed Dataset

In [ ]:
dropped_df['outliers'] = labels
noise_free_df = dropped_df[dropped_df['outliers'] == 0]
print("Noise free data has: " + str(len(noise_free_df)) + " rows")

In [ ]:
print('Highest Price: ' + str(max(noise_free_df['price'])))
print('Largest Lot:   ' + str(max(noise_free_df['lot_size'])))

After all processing before, we still have the noise like (price == 915,000,000, or lot_size == 4,356,000,000) which we do not want in our processed dataset. The reason for this may because of the inner work of DBSCAN, which is unsupervised clustering. This raw dataset may contain enough data for those extrodinary large values to become an individual cluster. 

### End of DBSCAN Method

## IQR (interquantile range) Method

#### Find IQR

In [ ]:
iqr_dataset = dropped_df[dropped_df.columns[4:11]]
Q1 = iqr_dataset.quantile(0.25)
Q3 = iqr_dataset.quantile(0.75)
IQR = Q3 - Q1
print(IQR)

In [ ]:
lower = Q1 - 1.5 * IQR
higer = Q3 + 1.5 * IQR

In [ ]:
iqr_dataset_out = iqr_dataset[~((iqr_dataset < (lower)) |(iqr_dataset > (higer))).any(axis=1)]

In [ ]:
iqr_df = raw_df.iloc[list(iqr_dataset_out.index),:]
iqr_df[['baths_half']] = iqr_df[['baths_half']].fillna(0)
iqr_df[['garage']] = iqr_df[['garage']].fillna(0)
print("IQR data has: " + str(len(iqr_df)) + " rows")

### Result Graphs for IQR (Blue is original data; Orange is processed data) 

#### Building size

In [ ]:
fig, axs = plt.subplots(2, 1)
axs[0].plot(np.sort(dropped_df['building_size']))
axs[1].plot(np.sort(iqr_dataset_out['building_size']), 'tab:orange')

#### Price

In [ ]:
fig, axs = plt.subplots(2, 1)
axs[0].plot(np.sort(dropped_df['price']))
axs[1].plot(np.sort(iqr_dataset_out['price']), 'tab:orange')

#### IQR method is considering all data as a whole. It is not based on any kind of clustering. Although we eliniminated a lot of extrodinary large values, we also eliniminated some reasonable data only because it is larger than the 75% quantile (price). 

### End of IQR method

### Overall, I think the data processed by IQR method is much more reasonable than the DBSCAN method, but either case need to be further adjusted to fulfill our estimatation.

## IQR Method By Zipcode

#### Total 7 categories to be evaluated

In [15]:
grouped_df = df.groupby("postal_code")

In [18]:
zip_iqr = 0
ori = 0
for num, group in grouped_df:
    iqr_dataset = group[group.columns[4:11]]
    Q1 = iqr_dataset.quantile(0.25)
    Q3 = iqr_dataset.quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    higher = Q3 + 1.5 * IQR
    iqr_dataset_out = iqr_dataset[~((iqr_dataset < (lower)) |(iqr_dataset > (higher))).any(axis=1)]
    print("IQR data has: " + str(len(iqr_dataset_out)) + " rows")
    zip_iqr += len(iqr_dataset_out)
    ori += len(iqr_dataset)
print(zip_iqr)
print(ori)

IQR data has: 38 rows
IQR data has: 73 rows
IQR data has: 62 rows
IQR data has: 91 rows
IQR data has: 46 rows
IQR data has: 35 rows
IQR data has: 24 rows
IQR data has: 50 rows
IQR data has: 8 rows
IQR data has: 56 rows
IQR data has: 27 rows
IQR data has: 18 rows
IQR data has: 8 rows
IQR data has: 60 rows
IQR data has: 102 rows
IQR data has: 25 rows
IQR data has: 76 rows
IQR data has: 126 rows
IQR data has: 45 rows
IQR data has: 2 rows
IQR data has: 55 rows
IQR data has: 20 rows
IQR data has: 160 rows
IQR data has: 145 rows
IQR data has: 122 rows
IQR data has: 130 rows
IQR data has: 20 rows
IQR data has: 37 rows
IQR data has: 39 rows
IQR data has: 94 rows
IQR data has: 20 rows
IQR data has: 80 rows
IQR data has: 84 rows
IQR data has: 58 rows
IQR data has: 63 rows
IQR data has: 35 rows
IQR data has: 128 rows
IQR data has: 36 rows
IQR data has: 82 rows
IQR data has: 111 rows
IQR data has: 103 rows
IQR data has: 106 rows
IQR data has: 105 rows
IQR data has: 149 rows
IQR data has: 102 rows


IQR data has: 134 rows
IQR data has: 129 rows
IQR data has: 94 rows
IQR data has: 93 rows
IQR data has: 100 rows
IQR data has: 90 rows
IQR data has: 64 rows
IQR data has: 108 rows
IQR data has: 91 rows
IQR data has: 81 rows
IQR data has: 78 rows
IQR data has: 94 rows
IQR data has: 46 rows
IQR data has: 101 rows
IQR data has: 95 rows
IQR data has: 113 rows
IQR data has: 131 rows
IQR data has: 121 rows
IQR data has: 128 rows
IQR data has: 15 rows
IQR data has: 135 rows
IQR data has: 52 rows
IQR data has: 83 rows
IQR data has: 70 rows
IQR data has: 61 rows
IQR data has: 3 rows
IQR data has: 13 rows
IQR data has: 69 rows
IQR data has: 82 rows
IQR data has: 93 rows
IQR data has: 77 rows
IQR data has: 41 rows
IQR data has: 65 rows
IQR data has: 166 rows
IQR data has: 170 rows
IQR data has: 7 rows
IQR data has: 90 rows
IQR data has: 30 rows
IQR data has: 28 rows
IQR data has: 84 rows
IQR data has: 60 rows
IQR data has: 47 rows
IQR data has: 100 rows
IQR data has: 42 rows
IQR data has: 32 rows

#### Only price to be evaluated

In [39]:
zip_iqr = 0
ori = 0
result_frame = []
for num, group in grouped_df:
    iqr_dataset = group[group.columns[8]]
    Q1 = iqr_dataset.quantile(0.25)
    Q3 = iqr_dataset.quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    higher = Q3 + 1.5 * IQR
    temp = group[(group["price"] >= lower) & (group["price"] <= higher)]
    zip_iqr += len(temp)
    ori += len(iqr_dataset)
    result_frame.append(temp)
result = pd.concat(result_frame)
print(zip_iqr)
print(ori)

82566
86806


In [43]:
for i in sorted(result['price']):
    print (i)

15000
25424
27000
27500
28000
28000
35000
45000
45000
47000
47500
50000
50000
50000
50000
53000
55000
57000
59000
59000
60000
60000
60000
60000
60000
62500
64000
64000
65000
65000
65000
65000
66000
67000
67000
67000
68000
68000
68500
70000
70000
70500
71000
72000
72000
72500
73000
74000
74500
75000
75000
75000
75000
75000
75000
75000
75000
76000
77500
78000
78000
80000
80000
80000
80000
80000
80000
80000
80500
81000
83000
85000
85000
85000
85000
85000
85000
87000
87000
87500
87500
87500
88000
88000
90000
90000
90000
90000
90000
90000
90500
90909
91000
91500
92000
92500
93000
93200
94500
95000
95000
95000
95000
95000
95000
95000
95000
95000
95000
95000
95100
96000
97000
97000
97000
97000
98000
98000
98500
99000
99000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
102000
102000
102500
102500
103500
104500
105000
105000
105000
1

245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245000
245400
245600
245900
246000
246000
246000
246000
246000
246000
246000
246500
247000
247000
247000
247000
247000
247000
247000
247000
247000
247000
247000
247000
247000
247000
247000
247000
247000
247000
247295
247500
247500
247500
247500
247500
247900
247950
248000
248000
248000
248000
248000
248000
248000
248000
248000
248000
248000
248000
248000
248000

300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000
300000

338000
338000
338000
338000
338000
338000
338000
338000
338000
338000
338000
338000
338000
338000
338000
338000
338000
338000
338000
338000
338000
338000
338000
338000
338000
338000
338000
338000
338000
338400
338500
338900
339000
339000
339000
339000
339000
339000
339000
339000
339000
339000
339000
339000
339000
339000
339000
339000
339000
339000
339000
339000
339000
339000
339000
339000
339000
339000
339000
339000
339000
339000
339000
339105
339500
339500
339500
339500
339700
339900
339900
339900
339950
339950
339999
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000
340000

370000
370000
370000
370000
370000
370000
370000
370000
370000
370000
370000
370000
370000
370000
370000
370000
370000
370000
370000
370000
370000
370000
370000
370000
370000
370000
370000
370000
370000
370000
370000
370000
370000
370000
370000
370000
370000
370000
370000
370200
370500
370500
370500
370500
370500
370500
370800
371000
371000
371000
371000
371000
371000
371000
371000
371000
371000
371000
371000
371350
371490
371490
371500
371500
372000
372000
372000
372000
372000
372000
372000
372000
372000
372000
372000
372000
372000
372000
372000
372000
372000
372000
372000
372000
372000
372000
372000
372000
372000
372000
372000
372000
372000
372000
372000
372000
372000
372000
372000
372000
372000
372020
372360
372494
372500
372500
372500
372500
372500
372500
372500
372500
372500
372500
372500
372500
372500
372500
372500
372500
372500
372500
372500
372999
373000
373000
373000
373000
373000
373000
373000
373000
373000
373000
373000
373000
373000
373000
373000
373000
373000
373000
373000

...

1105000
1105000
1105000
1105000
1105000
1105000
1105000
1105000
1105000
1105000
1105000
1105000
1105000
1105000
1105000
1105000
1105000
1105000
1105000
1105000
1105000
1105000
1105000
1105000
1105000
1105000
1105000
1105000
1105000
1105000
1105000
1105000
1105000
1105000
1105500
1106000
1106000
1106500
1107000
1107000
1107000
1107526
1108000
1108000
1108000
1108000
1108000
1108000
1108000
1108545
1109000
1109000
1109000
1109000
1109500
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110000
1110106
1110500
1110888
1111000
1111000
1111000


1250000
1250000
1250000
1250000
1250000
1250000
1250000
1250000
1250000
1250000
1250000
1250000
1250000
1250000
1250000
1250000
1250000
1250000
1250000
1250000
1250000
1250000
1250000
1250000
1250000
1250000
1250000
1250000
1250000
1250000
1250000
1250000
1250000
1250000
1250000
1250000
1250000
1250000
1250000
1250000
1250000
1250000
1250000
1250000
1250000
1250001
1250001
1250202
1250364
1250600
1251000
1251000
1251000
1251000
1251000
1251000
1251000
1251000
1251000
1251000
1252000
1252000
1252000
1252100
1252350
1252500
1253000
1254500
1254500
1255000
1255000
1255000
1255000
1255000
1255000
1255000
1255000
1255000
1255000
1255000
1255000
1255000
1255000
1255000
1255000
1255000
1255000
1255000
1255000
1255000
1255000
1255000
1255000
1255000
1255000
1255000
1255000
1255000
1255000
1255000
1255000
1255000
1255000
1255000
1255000
1255000
1255000
1255000
1255000
1255000
1255000
1255000
1255000
1255000
1255000
1255000
1255000
1255000
1255000
1255000
1255500
1256000
1256000
1256000
1256000


1450000
1450000
1450000
1450000
1450000
1450000
1450000
1450000
1450000
1450000
1450000
1450000
1450000
1450000
1450000
1450000
1450000
1450000
1450000
1450000
1450000
1450000
1450000
1450000
1450520
1451000
1451000
1451000
1451000
1451000
1451000
1451000
1451000
1452000
1452500
1452500
1452750
1455000
1455000
1455000
1455000
1455000
1455000
1455000
1455000
1455000
1455000
1455000
1455000
1455000
1455000
1455000
1455000
1455000
1455000
1455000
1455000
1455000
1455000
1455000
1455000
1455000
1455000
1455187
1455880
1456000
1456000
1456000
1456000
1456300
1457000
1457000
1457452
1458000
1458000
1458000
1458999
1459000
1459790
1460000
1460000
1460000
1460000
1460000
1460000
1460000
1460000
1460000
1460000
1460000
1460000
1460000
1460000
1460000
1460000
1460000
1460000
1460000
1460000
1460000
1460000
1460000
1460000
1460000
1460000
1460000
1460000
1460000
1460000
1460000
1460000
1460000
1460000
1460000
1460000
1460000
1460000
1460000
1460000
1460000
1460000
1460000
1460000
1460000
1460000


1715000
1715000
1715000
1715000
1715000
1715000
1715000
1715000
1715000
1715000
1715000
1715000
1715000
1716000
1716000
1716000
1716000
1717000
1718500
1720000
1720000
1720000
1720000
1720000
1720000
1720000
1720000
1720000
1720000
1720000
1720000
1720000
1720000
1720000
1720000
1720000
1720000
1720000
1720000
1720000
1720000
1720000
1720000
1720000
1720000
1720000
1720000
1720000
1721000
1721000
1722000
1722500
1722500
1722500
1723000
1724000
1724000
1724457
1725000
1725000
1725000
1725000
1725000
1725000
1725000
1725000
1725000
1725000
1725000
1725000
1725000
1725000
1725000
1725000
1725000
1725000
1725000
1725000
1725000
1725000
1725000
1725000
1725000
1725000
1725000
1725000
1725000
1725000
1725000
1725000
1725000
1725000
1725000
1725000
1725000
1725000
1725000
1725000
1725000
1725000
1725000
1725000
1725000
1725000
1725000
1725000
1725000
1725000
1725000
1725000
1725000
1725000
1725000
1725000
1725000
1725000
1726000
1726000
1726000
1727000
1727500
1728000
1728000
1728000
1728000


2458888
2460000
2460000
2460000
2460000
2460000
2460000
2460000
2460018
2465000
2465000
2465000
2467500
2468000
2470000
2470000
2470000
2470000
2470000
2470000
2470000
2471140
2472000
2475000
2475000
2475000
2475000
2475000
2475000
2475000
2475000
2475000
2475000
2475000
2475000
2475000
2475000
2475000
2475000
2475000
2475000
2475000
2480000
2480000
2480000
2480000
2480000
2480000
2480000
2485000
2485000
2485500
2488000
2488000
2488888
2489000
2490000
2490000
2490000
2490000
2495000
2495000
2495000
2495000
2495000
2495000
2495000
2495000
2495000
2495000
2497000
2498000
2499000
2499000
2499000
2499000
2499000
2500000
2500000
2500000
2500000
2500000
2500000
2500000
2500000
2500000
2500000
2500000
2500000
2500000
2500000
2500000
2500000
2500000
2500000
2500000
2500000
2500000
2500000
2500000
2500000
2500000
2500000
2500000
2500000
2500000
2500000
2500000
2500000
2500000
2500000
2500000
2500000
2500000
2500000
2500000
2500000
2500000
2500000
2500000
2500000
2500000
2500000
2500000
2500000


In [44]:
result.to_csv("IQR_housing_result.csv", header = True, index = False)